## Credit curves

* Just like a discount curve is a way of representing the underlying interest rates (or equivalently discount factors) implicit in the market quotes of a collection of real-world interest rate products, **credit curves** are a way of representing survival probabilities implied by credit default swaps.

* Credit default swaps (CDS) are instruments whose value depends on the likelihood that a given company (the curve's issuer) will suffer a credit event over a given period.

* A **credit event** can be a default, the failure to make payments, the issuer entering into bankruptcy proceedings, or the occurence of other legal events...
    * We will call a credit event a *default*;
    * and talk about **non-default probabilities** or survival probabilities($P_{\textrm{sur}}$), i.e. the probability that the issuer will not suffer a credit event before a given date 
        * *non-default probability is a cumulative probability* since refers to a time period. 

<table>
  <tr width=200>
    <th>Discount Curve</th>
    <th>Credit Curve</th>
  </tr>
  <tr>
    <td>Represents underlying rates implicit in market quotes of IR products</td>
    <td>Represents default probability implied by credit default swaps</td>
  </tr>
  <tr>
    <td>made of pillar_dates and discount factors</td>
    <td>made of pillar_dates and survival probabilities</td>
  </tr>
  <tr>
    <td>discount factors</td>
    <td>non-default probabilities</td>
  </tr>
  <tr>
    <td>short rate</td>
    <td>hazard rate</td>
  </tr>
</table>   

* Remember the short rate, $r_t$, is the interest rate at which an entity can borrow money for an infinitesimally short period of time from time $t$ to $t+dt$.

### Conditional Probability

* Hazard rate is often called a *conditional failure rate* since it's expression is a direct application of the conditional probability concept.

* Conditional probability answers to the question "how should you update probabilities of events when there is additional information available ?". 

#### Example

* Imagine a fair die is rolled: 
    * let $A$ be the event that the outcome is an odd number ($A={1,3,5}$); 
    * let $B$ be the event that the outcome is less than or equal to $3$ ($B={1,2,3}$). 
    1. What is the probability of $A$ ($P(A)$) ? 
    2. What is the probability of $A$ given $B$ ($P(A|B)$) ?

* Being a simple example we can compute the result by hand:

$$P(A) = \cfrac{|A|}{|S|} = \cfrac{|\{1,3,5\}|}{6} = \cfrac{1}{2}\qquad\textrm{(where $S$ is the entire sample space)}$$

* If $B$ has occurred, the outcome must be among $\{1,2,3\}$. 

* For $A$ to also happen the outcome must be in $A\cap B = \{1,3\}$. 
    * Since the die is fair, we argue that $P(A|B)$ must be equal to

$$P(A|B) = \cfrac{|A\cap B|}{|B|} = \cfrac{2}{3}$$

* To generalize, divide numerator and denominator by the entire space of the events $|S|$ hence:

$$P(A|B) = \cfrac{|A\cap B|}{|B|} = \cfrac{\cfrac{|A\cap B|}{|S|}}{\cfrac{|B|}{|S|}} = \cfrac{P(A\cap B)}{P(B)}$$

<img src="https://drive.google.com/uc?id=16bF8jTwIMLPPDK2vIP99CV-RcBJTocyk" width=500>

### Hazard Rate
* Hazard rate represents the instantaneous probability of the issuer defaulting *conditioned* on it not having defaulted until that moment.

$$\lambda(t) = \cfrac{\cfrac{P_{\textrm{def}}(\tau \in (t, t+dt))}{dt}}{P_{\textrm{def}}(\tau\gt t)} = \cfrac{\cfrac{dP_{\textrm{def}}}{dt}}{P_{\textrm{sur}}(t_0, t)} = \cfrac{\cfrac{d(1-P_{\textrm{sur}})}{dt}}{P_{\textrm{sur}}(t_0, t)} = -\cfrac{dP_{\textrm{sur}}}{dt}\cfrac{1}{P_{\textrm{sur}}(t_0, t)}$$

* where the default (survival) probability is indicated by $P_{\textrm{def}}$ ($P_{\textrm{sur}}$), the hazard rate by $\lambda$ and the time of default with $\tau$. The minus sign derives from the fact that $P_{\textrm{sur}}$ is a **non** default probability while the hazard rate is defined in terms of the probability of default $P_{\textrm{def}}$.

* Given the hazard rate the survival probability can be determined as:

$$\lambda(t) = -\cfrac{1}{dt}\cdot\cfrac{dP_{\textrm{sur}}}{P_{\textrm{sur}}} = -\cfrac{d(\textrm{log}P_{\textrm{sur}})}{dt}$$

$$P_{\textrm{sur}}(t_0, t) = e^{-\int_{t_0}^{t}\lambda(s) ds}$$

### CreditCurve class

In [2]:
# implement CreditCurve class
from numpy import interp
from dateutil.relativedelta import relativedelta

class CreditCurve:
    def __init__(self, pillar_dates, ndps):
        self.start_date = pillar_dates[0]
        self.pillar_days = [(pd - self.start_date).days for pd in pillar_dates]
        self.ndps = ndps
    
    def ndp(self, value_date):
        value_days = (value_date - self.start_date).days
        return interp(value_days, self.pillar_days, self.ndps)

    def hazard(self, value_date):
        ndp_1 = self.ndp(value_date)
        ndp_2 = self.ndp(value_date + relativedelta(days=1))
        delta_t = 1.0 / 365.0
        h = -1.0 / ndp_1 * (ndp_2 - ndp_1) / delta_t
        return h

* As usual we test the newly developed class with some data.

In [6]:
# set observation_date and CreditCurve
from datetime import date
from dateutil.relativedelta import relativedelta

start_date = date.today()
cc = CreditCurve(
    [start_date, start_date + relativedelta(years=2)],
    [1.0, 0.8])

In [5]:
# check ndp in 1 year
cc.ndp(start_date + relativedelta(years=1))

0.9

In [7]:
# check hazard in 1 year
cc.hazard(start_date + relativedelta(years=1))

0.11111111111112416

## Credit Deafult Swaps

* A Credit Default Swap (CDS) is a financial swap agreement that the seller of the CDS will compensate the buyer in the event of a credit event. 
    * The seller of the CDS insures the buyer against some reference asset defaulting. 
    * The buyer of the CDS makes a series of payments (the CDS "fee" or "spread") to the seller and, in exchange, may expect to receive a payoff if the asset defaults. 

* CDSs are made up of two legs:
    * the *default* leg: which pays $LGD = F(1 - R)$, known as the **loss given default**, if and when the credit event occurs, $F$ is the face value of the contact, $R$ is the recovery rate (usually set around 40%);
    * the *premium* leg: which pays the *spread* $S$ every m months until the credit event occurs.

### Premium leg

Let's start with the premium leg. We will use the following notation:

* $d$ today's date;
* $d_0$ the start date of the CDS (could be different from $d$);
* $d_1, ..., d_n$ the payment dates of the premium leg, which occur at a m-month frequency (we assume that $d_n$ is the end date of the CDS);
* $D(d')$ the discount factor between $d$ and $d'$;
* $P_{\textrm{sur}}(d')$ the survival probability between $d$ and $d'$;
* $\tau$ the random variable representing the date of the credit event.

At each payment date $d_i$, a flow $F\cdot S$ is paid if and only if the credit event has not occurred before that date. Since the NPV depends on the default probability, the value of the premium leg can be estimated by an expectation

$$f_{\textrm{premium}}^i = \mathbb{E}\left[ S \times D(d_i) \times \mathbb{1}(\tau > d_i) \right]$$

where $\mathbb{1}(\tau > d_i)$ means that the expectation value has to be evaluated when $\tau > d_i$. Remember that if $x$ is a random variable with a finite number of finite outcomes $x_{1},x_{2},\ldots ,x_{k}$ occurring with probabilities $p_{1},p_{2},\ldots ,p_{k}$ respectively, the expectation of $x$ is:

$$\mathbb{E}[x] = \sum _{i=1}^{k}x_{i}\,p_{i}=x_{1}p_{1}+x_{2}p_{2}+\cdots +x_{k}p_{k}$$
which is the weighted sum of the $x_i$, with $p_{i}$ values being the weights. 

In our case $x_i = S\cdot D(d_i)$ and $p_i=P_{\textrm{sur}}(d_i)$ so the NPV of the leg can be expressed as:

$$\textrm{NPV}_{premium} = F\cdot S\cdot \sum_{i=1}^{n} D(d_i) \cdot P_{\textrm{sur}}(d_i)$$

## Default leg

The LGD is paid out on the same date on which the credit event occurs, i.e. it can potentially be paid out on any date between $d_0$ and $d_n$. Mathematically, therefore, the NPV of the premium leg can be expressed as follows:

$$\mathrm{NPV_{default}} = \mathbb{E} \left[F(1-R) \times D(\tau) \times \mathbb{1}(\tau \leq d_n) \right] $$

Using the laws of probability, we can break this down into the sum of "daily NPVs" calculated as a function of the daily default probabilities $P_{\textrm{def}}$:

$$
\begin{align*}
\mathbb{E}\left[F(1-R) \times D(\tau) \times \mathbb{1}(\tau \leq d_n) \right]
&= \sum_{d'=d_0}^{d_n} \mathbb{E}[ F(1-R) \times D(\tau) | \tau = d'] P_{\textrm{def}}[ \tau = d' ] \\
&= F(1-R) \sum_{d'=d_0}^{d_n} D(d') \left( P_{\textrm{def}}[ \tau \geq d' ] - P_{\textrm{def}}[ \tau \geq d'+1 ] \right) \\
&= F(1-R) \sum_{d'=d_0}^{d_n} D(d') \left( P_{\textrm{sur}}(d') - P_{\textrm{sur}}(d'+1) \right)
\end{align*}
$$

where the last step holds since $P_{\textrm{def}}[\tau\geq d'] = 1 - P_{\textrm{def}}[\tau < d'] = 1 - (1-P_{\textrm{sur}}(\tau < d')) = P_{\textrm{sur}}(\tau < d')$.

<img src="timeline.png">

In [69]:
# credit default swap class with breakeven method
from finmarkets import generate_dates

class CreditDefaultSwap:
    def __init__(self, notional, start_date, fixed_spread,
                 maturity_y, tenor=3, recovery=0.4):
        self.notional = notional
        self.payment_dates = generate_dates(start_date,
                                            maturity_y*12, tenor)
        self.fixed_spread = fixed_spread
        self.recovery = recovery

    def npv_premium_leg(self, discount_curve, credit_curve):
        npv = 0
        for i in range(1, len(self.payment_dates)):
            npv += (self.fixed_spread *
                    discount_curve.df(self.payment_dates[i]) *
                    credit_curve.ndp(self.payment_dates[i]))
        return npv * self.notional

    def npv_default_leg(self, discount_curve, credit_curve):
        npv = 0
        d = self.payment_dates[0]
        while d <= self.payment_dates[-1]:
            npv += discount_curve.df(d) * (
                   credit_curve.ndp(d) -
                   credit_curve.ndp(d + relativedelta(days=1)))
            d += relativedelta(days=1)
        return npv * self.notional * (1 - self.recovery)

    def npv(self, discount_curve, credit_curve):
        return self.npv_default_leg(discount_curve, credit_curve) - \
               self.npv_premium_leg(discount_curve, credit_curve)

* Below a simple test of the class, using [discount_curve.xlsx](https://github.com/matteosan1/finance_course/blob/develop/libro/input_files/discount_curve.xlsx?raw=true).

In [72]:
# test CDS class with previous inputs 
import pandas as pd
from finmarkets import DiscountCurve
from dateutil.relativedelta import relativedelta
from datetime import date

dc_data = pd.read_excel("https://github.com/matteosan1/finance_course/blob/develop/libro/input_files/discount_curve.xlsx?raw=true")
observation_date = date.today() 
dc = DiscountCurve(observation_date,
dc_data['pillars'].dt.date.tolist(),
dc_data['discount_factors'].tolist())
credit_curve = CreditCurve([start_date,
                            start_date + relativedelta(months=36)],
                            [1.0, 0.7])
cds = CreditDefaultSwap(1e6, start_date, 0.03, 3)

In [73]:
# check default leg, premium leg and npv
print (cds.npv_premium_leg(dc, credit_curve))
print (cds.npv_default_leg(dc, credit_curve))

303160.9661537263
180949.18879529936


In [74]:
cds.npv(dc, credit_curve)

-122211.77735842692

## Estimate Default Probabilities from CDS

* We can estimate default probabilities (hence credit curves) from CDS quotes using *bootstrap*:
    1. collect market quotes for a number of CDS with different maturities;
    2. create the corresponding CDS objects;
    3. define a $\tt{CreditCurve}$ whose pillars are the CDS maturity dates and the survival probabilities are unknown;
    4. define an objective function to minimize the sum of the squared CDS's NPVs;
    5. set the non-default probabilities to an initial value and define their range of variability between $[0, 1]$ since they are probabilities and fix "today's" probability to 1 since there hasn't been any default;
    6. run the minimization.

* The file with a test set of market quotes is [cds_quotes.xlsx](https://github.com/matteosan1/finance_course/raw/develop/libro/input_files/cds_quotes.xlsx).


In [80]:
# bootstrapping for CDS
from scipy.optimize import minimize
import pandas as pd
import numpy as np

dc = pd.read_excel("https://github.com/matteosan1/finance_course/blob/develop/libro/input_files/discount_curve.xlsx?raw=true")
mq = pd.read_excel("https://github.com/matteosan1/finance_course/raw/develop/libro/input_files/cds_quotes.xlsx")
discount_curve = DiscountCurve(observation_date,
                               dc['pillars'].dt.date.tolist(),
                               dc['discount_factors'].tolist())
cdswaps = []
pillar_dates = [start_date]
for i in range(len(mq)):
    cds = CreditDefaultSwap(1e6, start_date,
                            mq['quote'].tolist()[i],
                            mq['months'].tolist()[i]//12)
    cdswaps.append(cds)
    pillar_dates.append(cds.payment_dates[-1])

def objective_function(unknown_ndps):
    unknown_ndps = np.insert(unknown_ndps, 0, 1)
    credit_curve = CreditCurve(pillar_dates, unknown_ndps)
    sum_sq = 0
    for cds in cdswaps:
        sum_sq += cds.npv(discount_curve, credit_curve)**2
    return sum_sq

ndp_guess = [1 for _ in range(len(pillar_dates)-1)]
bounds = [(0.01, 1) for _ in range(len(pillar_dates)-1)]

r = minimize(objective_function, ndp_guess, bounds=bounds)

In [81]:
print (r)

      fun: 1.6567237922097584e-05
 hess_inv: <6x6 LbfgsInvHessProduct with dtype=float64>
      jac: array([1057.6428894 ,  697.66468945, 2288.37362133, 3207.19521611,
       3960.08247585, 3403.59384113])
  message: 'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 140
      nit: 11
     njev: 20
   status: 0
  success: True
        x: array([0.90794559, 0.80382963, 0.70844196, 0.48031869, 0.29174604,
       0.06689759])


In [83]:
x = np.insert(r.x, 0, 1)
cc = CreditCurve(pillar_dates, x)
for i in range(len(pillar_dates)):
    print ("S(t<{}): {:.2f}".format(pillar_dates[i], x[i]))

S(t<2021-10-12): 1.00
S(t<2022-10-12): 0.91
S(t<2023-10-12): 0.80
S(t<2024-10-12): 0.71
S(t<2027-10-12): 0.48
S(t<2031-10-12): 0.29
S(t<2041-10-12): 0.07


### Determine Default Probabilities from Bond Prices

* The price of a bond is directly linked to the credit rating of the issuer.
    * There is always an associated default risk: the borrower might not be able to repay the loan.
    * Bonds with low ratings (*junk bonds*) are sold at lower prices since riskier;
    * high ratings bonds (*investment-grade bonds*) are sold at higher prices.


* The average default intensity can be approximated by considering the spread $s$ between a risky bond yield and the risk-free rate, and $R$ the recovery rate. 
    * The expected loss due to default can be expressed both in terms of spread and loss given default:
    
$$s \Delta t \approx \lambda\Delta t (1 − R) \implies \bar{\lambda} = \cfrac{s}{(1-R)}$$


* Imagine for example a bond that yields 150 bp more than a similar risk-free bond and assume an expected recovery rate of 40%.

$$\bar{\lambda} = \cfrac{0.015}{(1-0.4)} = 2.5\%$$